In [ ]:
!wget dataset_ozon.tsv https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv
!wget https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz

--2023-03-22 08:29:19--  http://dataset_ozon.tsv/
Resolving dataset_ozon.tsv (dataset_ozon.tsv)... failed: Name or service not known.
wget: unable to resolve host address ‘dataset_ozon.tsv’
--2023-03-22 08:29:19--  https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9640627 (9.2M) [text/plain]
Saving to: ‘dataset_ozon.tsv’

dataset_ozon.tsv    100%[===================>]   9.19M  --.-KB/s    in 0.06s   

2023-03-22 08:29:19 (162 MB/s) - ‘dataset_ozon.tsv’ saved [9640627/9640627]

FINISHED --2023-03-22 08:29:19--
Total wall clock time: 0.5s
Downloaded: 1 files, 9.2M in 0.06s (162 MB/s)
--2023-03-22 08:29:19--  https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz
Resolving 

In [ ]:
!pip install nerus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from nerus import load_nerus
from tqdm import tqdm
from collections import Counter
from nltk.tokenize import wordpunct_tokenize
import nltk
nltk.download('punkt')
tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
ds = pd.read_csv('/content/dataset_ozon.tsv', sep = '\t', encoding='utf-8')
ds = ds.drop(columns = ['Unnamed: 0'])

In [ ]:
docs = load_nerus('/content/nerus_lenta.conllu.gz')

In [ ]:
def make_tags(n, b):
    global tags
    ner_tags_id = []
    name = wordpunct_tokenize(n)
    brand = wordpunct_tokenize(b)
    results=[]
    sll=len(brand)
    for ind in (i for i,e in enumerate(name) if e==brand[0]):
        if name[ind:ind+sll]==brand:
            results.append((ind,ind+sll-1))
    if results != []:
      ner_tags_labels = ['O'] * len(name)    
      for result in results:
          if result[0] == result[1]:
              ner_tags_labels[result[0]] = 'B-BRAND'
          else:
            ner_tags_labels[result[0]] = 'B-BRAND'
            for j in range(result[0]+1, result[1]+1):
              ner_tags_labels[j] = 'I-BRAND'

      tags.append({'tokens': name, 'ner_tags_labels': ner_tags_labels})

In [ ]:
tags = []
ds.progress_apply(lambda x: make_tags(x['name'], x['brand']), axis = 1)

100%|██████████| 81779/81779 [00:03<00:00, 22315.19it/s]


0        None
1        None
2        None
3        None
4        None
         ... 
81774    None
81775    None
81776    None
81777    None
81778    None
Length: 81779, dtype: object

In [ ]:
def find_in_nerus(docs, tag):
  global result
  count_tag = 0
  tags2id = {'B-PER':3, 'I-PER': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-ORG': 7, 'I-ORG': 8, 'O': 0}
  b_tag = 'B-' + tag
  i_tag = 'I-' + tag
  for doc in docs:
      if count_tag >= len(tags):
          return 'Done'
      for sent in doc.sents:
          tokens = []
          ner_tags_labels = []
          for token in sent.tokens:
              tokens.append(token.text)
              ner_tags_labels.append(token.tag)
          c = Counter(ner_tags_labels)
          count_tag = count_tag + c[b_tag] + c[i_tag]
          if c[b_tag] + c[i_tag] > 0:
              if {'tokens': tokens, 'ner_tags_labels': ner_tags_labels} not in result:
                  result.append({'tokens': tokens, 'ner_tags_labels': ner_tags_labels})

In [ ]:
result = []
all_tag = ['PER', 'LOC', 'ORG']
for tag in tqdm(all_tag):
    find_in_nerus(docs, tag)

100%|██████████| 3/3 [15:00<00:00, 300.17s/it]


In [ ]:
full_list = result + tags
ds_IOB2 = pd.DataFrame(full_list, columns = ['tokens', 'ner_tags_labels'])

In [ ]:
!pip install datasets
!pip install transformers datasets evaluate seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.7 MB/s eta 0:00

In [ ]:
from datasets import Dataset, Features, Value, ClassLabel,  Sequence
ner_tags_labels_ = ['O', 'B-BRAND', 'I-BRAND', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']
ds_features = Features({'tokens':  Sequence(Value("string")),
                        'ner_tags_labels': Sequence(ClassLabel(names=ner_tags_labels_))})

dataset = Dataset.from_pandas(ds_IOB2, features=ds_features)
dataset_splitted = dataset.train_test_split(test_size=0.2, seed=88)

In [ ]:
dataset

Dataset({
    features: ['tokens', 'ner_tags_labels'],
    num_rows: 191250
})

In [ ]:
import numpy as np
import torch


from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import pipeline

In [ ]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids: 
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
ds_tokenized = dataset_splitted.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/153000 [00:00<?, ? examples/s]

Map:   0%|          | 0/38250 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
seqeval = evaluate.load("seqeval")
label_list = ds_tokenized["train"].features["ner_tags_labels"].feature.names

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {i: label_list[i] for i in range(len(label_list))}
label2id = {label_list[i]: i for i in range(len(label_list))}

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [ ]:
training_args = TrainingArguments(
    output_dir="xlm_roberta_base_ner_with_brand_rus",
    learning_rate=4e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=5000,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tokenized["train"],
    eval_dataset=ds_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1000,0.057000,0.046072,0.913700,0.923813,0.918729,0.985773
2000,0.044400,0.040830,0.933801,0.945564,0.939646,0.989112
3000,0.038800,0.036706,0.929644,0.954154,0.941740,0.989147
4000,0.034500,0.032501,0.946364,0.950534,0.948444,0.990971
5000,0.035800,0.028745,0.951578,0.956929,0.954246,0.991627
6000,0.031800,0.029101,0.945412,0.960252,0.952774,0.991486
7000,0.029700,0.026723,0.952342,0.957641,0.954984,0.991981
8000,0.028500,0.024285,0.957178,0.959763,0.958469,0.992573
9000,0.026800,0.023893,0.959437,0.958056,0.958746,0.992623
10000,0.019700,0.024968,0.963229,0.958027,0.960621,0.992650


TrainOutput(global_step=19126, training_loss=0.029171142308367703, metrics={'train_runtime': 8697.2816, 'train_samples_per_second': 35.183, 'train_steps_per_second': 2.199, 'total_flos': 1.030109251296528e+16, 'train_loss': 0.029171142308367703, 'epoch': 2.0})

In [ ]:
from transformers import Pipeline
class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "tokenizer" in kwargs:
            preprocess_kwargs["tokenizer"] = kwargs["tokenizer"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, text):
        self.text_splt = wordpunct_tokenize(text)
        self.tokenized = self.tokenizer(self.text_splt, is_split_into_words=True, return_tensors='pt')
        return self.tokenized

    def _forward(self, model_inputs):
        model_inputs['input_ids'] = model_inputs['input_ids'].to('cuda')
        model_inputs['attention_mask'] = model_inputs['attention_mask'].to('cuda')
        return self.model(**model_inputs)

    def postprocess(self, model_outputs):
        tokens = self.tokenizer.convert_ids_to_tokens(list(self.tokenized["input_ids"][0]))
        res = model_outputs[0].numpy()[0]
        clses = []
        for r in res:
            class_for_token = np.argmax(r, axis=0)
            clses.append(class_for_token)
        for j in range(len(tokens)):
            if clses[j] > 0:
              clses[j] = tokens[j]
        result = []
        for y in clses:
            if y !=0 and not y.startswith('_'):
                result.append(str(y))
            else:
                result.append(' ' + str(y))
        x = ''.join(str(elem) for elem in result)
        x = x.replace('▁', ' ')
        x = x.replace('<s>', '')
        x = x.split('0')
        x = [o[1:-1] for o in x if o != ' ' and o != '']
        return x

pipeline = MyPipeline(model=model.to('cuda'), tokenizer=tokenizer)
pipeline('Кофеварка Polaris PCM 1535E Adore Cappuccino')

['Polaris']

In [ ]:
pipeline('Молоко стерилизованное Домик в деревне 2.5% 950г')

['Домик в деревне']

In [ ]:
pipeline('Вода негазированная Святой Источник, со вкусом лимона, 0,5 л')

['Святой Источник']

In [ ]:
pipeline('Вода негазированная святой источник, со вкусом лимона, 0,5 л')

['святой источник']